# DataCo Smart Supply Chain Model Testing

## Importing important library

In [45]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score,r2_score,accuracy_score,classification_report,confusion_matrix, f1_score

## Seeing the data structures

In [2]:
df = pd.read_csv('DataCoSupplyChainDataset.csv', encoding='unicode_escape')
pd.set_option('display.max_columns', None)
df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725.0,2,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,20755,1/31/2018 22:56,77202,1360,13.110000,0.04,180517,327.75,0.29,1,327.75,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,19492,Luna,XXXXXXXXX,Consumer,PR,2679 Rustic Loop,725.0,2,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,19492,1/13/2018 12:27,75939,1360,16.389999,0.05,179254,327.75,-0.80,1,327.75,311.359985,-249.089996,South Asia,Rajastán,PENDING,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,19491,Maldonado,XXXXXXXXX,Consumer,CA,8510 Round Bear Gate,95125.0,2,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,19491,1/13/2018 12:06,75938,1360,18.030001,0.06,179253,327.75,-0.80,1,327.75,309.720001,-247.779999,South Asia,Rajastán,CLOSED,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,EE. UU.,XXXXXXXXX,Tana,19490,Tate,XXXXXXXXX,Home Office,CA,3200 Amber Bend,90027.0,2,Fitness,34.125946,-118.291016,Pacific Asia,Townsville,Australia,19490,1/13/2018 11:45,75937,1360,22.940001,0.07,179252,327.75,0.08,1,327.75,304.809998,22.860001,Oceania,Queensland,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Orli,19489,Hendricks,XXXXXXXXX,Corporate,PR,8671 Iron Anchor Corners,725.0,2,Fitness,18.253769,-66.037048,Pacific Asia,Townsville,Australia,19489,1/13/2018 11:24,75936,1360,29.500000,0.09,179251,327.75,0.45,1,327.75,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [3]:
num_feat = [n for n in df.columns if df[n].dtypes!='O']
cat_feat = [c for c in df.columns if df[c].dtypes=='O']
print('Number of numerical features: ', len(num_feat))
print('Number of categorical features: ', len(cat_feat))

Number of numerical features:  29
Number of categorical features:  24


In [4]:
num_feat

['Days for shipping (real)',
 'Days for shipment (scheduled)',
 'Benefit per order',
 'Sales per customer',
 'Late_delivery_risk',
 'Category Id',
 'Customer Id',
 'Customer Zipcode',
 'Department Id',
 'Latitude',
 'Longitude',
 'Order Customer Id',
 'Order Id',
 'Order Item Cardprod Id',
 'Order Item Discount',
 'Order Item Discount Rate',
 'Order Item Id',
 'Order Item Product Price',
 'Order Item Profit Ratio',
 'Order Item Quantity',
 'Sales',
 'Order Item Total',
 'Order Profit Per Order',
 'Order Zipcode',
 'Product Card Id',
 'Product Category Id',
 'Product Description',
 'Product Price',
 'Product Status']

In [5]:
cat_feat

['Type',
 'Delivery Status',
 'Category Name',
 'Customer City',
 'Customer Country',
 'Customer Email',
 'Customer Fname',
 'Customer Lname',
 'Customer Password',
 'Customer Segment',
 'Customer State',
 'Customer Street',
 'Department Name',
 'Market',
 'Order City',
 'Order Country',
 'order date (DateOrders)',
 'Order Region',
 'Order State',
 'Order Status',
 'Product Image',
 'Product Name',
 'shipping date (DateOrders)',
 'Shipping Mode']

In [6]:
df=df.drop('Customer Password', axis=1)
df=df.drop('Customer Email', axis=1)

In [7]:
date = ['shipping date (DateOrders)', 'order date (DateOrders)']

In [8]:
df['shipping date (DateOrders)'] = pd.to_datetime(df['shipping date (DateOrders)'])
df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'])

Choosing Columns for Late Delivery Risk Prediction

In [10]:
late_pred = ['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)', 'Delivery Status', 'Late_delivery_risk',
            'Shipping Mode','Benefit per order', 'Sales per customer','Latitude','Longitude','Order Status','Order Region',
            'Order Country','Order City','Market', 'shipping date (DateOrders)', 'order date (DateOrders)']

In [11]:
df_ship =df[late_pred]
df_ship.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Delivery Status,Late_delivery_risk,Shipping Mode,Benefit per order,Sales per customer,Latitude,Longitude,Order Status,Order Region,Order Country,Order City,Market,shipping date (DateOrders),order date (DateOrders)
0,DEBIT,3,4,Advance shipping,0,Standard Class,91.250000,314.640015,18.251453,-66.037056,COMPLETE,Southeast Asia,Indonesia,Bekasi,Pacific Asia,2018-02-03 22:56:00,2018-01-31 22:56:00
1,TRANSFER,5,4,Late delivery,1,Standard Class,-249.089996,311.359985,18.279451,-66.037064,PENDING,South Asia,India,Bikaner,Pacific Asia,2018-01-18 12:27:00,2018-01-13 12:27:00
2,CASH,4,4,Shipping on time,0,Standard Class,-247.779999,309.720001,37.292233,-121.881279,CLOSED,South Asia,India,Bikaner,Pacific Asia,2018-01-17 12:06:00,2018-01-13 12:06:00
3,DEBIT,3,4,Advance shipping,0,Standard Class,22.860001,304.809998,34.125946,-118.291016,COMPLETE,Oceania,Australia,Townsville,Pacific Asia,2018-01-16 11:45:00,2018-01-13 11:45:00
4,PAYMENT,2,4,Advance shipping,0,Standard Class,134.210007,298.250000,18.253769,-66.037048,PENDING_PAYMENT,Oceania,Australia,Townsville,Pacific Asia,2018-01-15 11:24:00,2018-01-13 11:24:00


In [12]:
df_ship.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 17 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Type                           180519 non-null  object        
 1   Days for shipping (real)       180519 non-null  int64         
 2   Days for shipment (scheduled)  180519 non-null  int64         
 3   Delivery Status                180519 non-null  object        
 4   Late_delivery_risk             180519 non-null  int64         
 5   Shipping Mode                  180519 non-null  object        
 6   Benefit per order              180519 non-null  float64       
 7   Sales per customer             180519 non-null  float64       
 8   Latitude                       180519 non-null  float64       
 9   Longitude                      180519 non-null  float64       
 10  Order Status                   180519 non-null  object        
 11  

In [13]:
df_ship['ship_date'] = pd.DatetimeIndex(df['shipping date (DateOrders)']).day
df_ship['ship_month'] = pd.DatetimeIndex(df['shipping date (DateOrders)']).month
df_ship['ship_year'] = pd.DatetimeIndex(df['shipping date (DateOrders)']).year
df_ship['order_date'] = pd.DatetimeIndex(df['shipping date (DateOrders)']).day
df_ship['order_month'] = pd.DatetimeIndex(df['shipping date (DateOrders)']).month
df_ship['order_year'] = pd.DatetimeIndex(df['shipping date (DateOrders)']).year

In [14]:
df_ship.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Delivery Status,Late_delivery_risk,Shipping Mode,Benefit per order,Sales per customer,Latitude,Longitude,Order Status,Order Region,Order Country,Order City,Market,shipping date (DateOrders),order date (DateOrders),ship_date,ship_month,ship_year,order_date,order_month,order_year
0,DEBIT,3,4,Advance shipping,0,Standard Class,91.250000,314.640015,18.251453,-66.037056,COMPLETE,Southeast Asia,Indonesia,Bekasi,Pacific Asia,2018-02-03 22:56:00,2018-01-31 22:56:00,3,2,2018,3,2,2018
1,TRANSFER,5,4,Late delivery,1,Standard Class,-249.089996,311.359985,18.279451,-66.037064,PENDING,South Asia,India,Bikaner,Pacific Asia,2018-01-18 12:27:00,2018-01-13 12:27:00,18,1,2018,18,1,2018
2,CASH,4,4,Shipping on time,0,Standard Class,-247.779999,309.720001,37.292233,-121.881279,CLOSED,South Asia,India,Bikaner,Pacific Asia,2018-01-17 12:06:00,2018-01-13 12:06:00,17,1,2018,17,1,2018
3,DEBIT,3,4,Advance shipping,0,Standard Class,22.860001,304.809998,34.125946,-118.291016,COMPLETE,Oceania,Australia,Townsville,Pacific Asia,2018-01-16 11:45:00,2018-01-13 11:45:00,16,1,2018,16,1,2018
4,PAYMENT,2,4,Advance shipping,0,Standard Class,134.210007,298.250000,18.253769,-66.037048,PENDING_PAYMENT,Oceania,Australia,Townsville,Pacific Asia,2018-01-15 11:24:00,2018-01-13 11:24:00,15,1,2018,15,1,2018


In [15]:
print(df['Shipping Mode'].value_counts())
print(df['Order Status'].value_counts())

Standard Class    107752
Second Class       35216
First Class        27814
Same Day            9737
Name: Shipping Mode, dtype: int64
COMPLETE           59491
PENDING_PAYMENT    39832
PROCESSING         21902
PENDING            20227
CLOSED             19616
ON_HOLD             9804
SUSPECTED_FRAUD     4062
CANCELED            3692
PAYMENT_REVIEW      1893
Name: Order Status, dtype: int64


In [18]:
df_ship['Order Country'].value_counts()

Estados Unidos       24840
Francia              13222
México               13172
Alemania              9564
Australia             8497
                     ...  
Kuwait                   2
Eritrea                  2
Guinea Ecuatorial        2
Burundi                  1
Serbia                   1
Name: Order Country, Length: 164, dtype: int64

In [19]:
df_ship = df_ship.drop(['Days for shipping (real)','Days for shipment (scheduled)','Order City'], axis=1)

## Data Preprocessing

In [20]:
cleanup = {'Shipping Mode':{"Standard Class":0,"Second Class" : 1,"First Class":2,"Same Day":3}}

df_ship=df_ship.replace(cleanup)
df_ship.head(2)

,Type,Delivery Status,Late_delivery_risk,Shipping Mode,Benefit per order,Sales per customer,Latitude,Longitude,Order Status,Order Region,Order Country,Market,shipping date (DateOrders),order date (DateOrders),ship_date,ship_month,ship_year,order_date,order_month,order_year
0,DEBIT,Advance shipping,0,0,91.250000,314.640015,18.251453,-66.037056,COMPLETE,Southeast Asia,Indonesia,Pacific Asia,2018-02-03 22:56:00,2018-01-31 22:56:00,3,2,2018,3,2,2018
1,TRANSFER,Late delivery,1,0,-249.089996,311.359985,18.279451,-66.037064,PENDING,South Asia,India,Pacific Asia,2018-01-18 12:27:00,2018-01-13 12:27:00,18,1,2018,18,1,2018


In [30]:
df_ship = df_ship.drop(['shipping date (DateOrders)','order date (DateOrders)'], axis=1)

In [31]:
print(df_ship.shape)
df_ship = pd.get_dummies(df_ship)
df_ship.shape

(180519, 221)


(180519, 221)

In [32]:
from sklearn.model_selection import train_test_split

y=df_ship['Late_delivery_risk']
X=df_ship.drop('Late_delivery_risk',axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

logreg = LogisticRegression()
gnb = GaussianNB()
rfc = RandomForestClassifier()
tree = DecisionTreeClassifier()

## Simple Model Testing and Validation

In [44]:
model = [logreg, gnb, rfc, tree]

for mod in model:
    mod.fit(X_train, y_train)
    pred = mod.predict(X_valid)
    a = accuracy_score(y_valid, pred)
    b = r2_score(y_valid, pred)
    c = f1_score(y_valid, pred)
    d = roc_auc_score(y_valid, pred)
    e = classification_report(y_valid, pred)
    f = confusion_matrix(y_valid, pred)
    print('Accuracy Score ', mod, ' : ', a)
    print('R2_Score Score ', mod, ' : ', b)
    print('f1_Score Score ', mod, ' : ', c)
    print('ROC AUC ', mod, ' : ', d)
    print('Classification report ', mod, ' : ', e)
    print('Confusion matrix ', mod, ' :', '\n\n', f)
    print('-'*75)

Accuracy Score  LogisticRegression()  :  0.9628849988920896
R2_Score Score  LogisticRegression()  :  0.8502615721665822
f1_Score Score  LogisticRegression()  :  0.9671391436558929
ROC AUC  LogisticRegression()  :  0.959111607921051
Classification report  LogisticRegression()  :                precision    recall  f1-score   support

           0       1.00      0.92      0.96     16384
           1       0.94      1.00      0.97     19720

    accuracy                           0.96     36104
   macro avg       0.97      0.96      0.96     36104
weighted avg       0.97      0.96      0.96     36104

Confusion matrix  LogisticRegression()  : 

 [[15045  1339]
 [    1 19719]]
---------------------------------------------------------------------------
Accuracy Score  GaussianNB()  :  1.0
R2_Score Score  GaussianNB()  :  1.0
f1_Score Score  GaussianNB()  :  1.0
ROC AUC  GaussianNB()  :  1.0
Classification report  GaussianNB()  :                precision    recall  f1-score   support

     